<h3 align=center>After gaining knowledge on Logistic Regression section on '365 Data Science', I have created a model to predict the chances of someone getting a heart attack. Dataset downloaded from kaggle.com<h3>

In [1]:
##Import relevant libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
sns.set()

## Load the data

In [2]:
raw_data = pd.read_csv("framingham_heart_disease.csv")
raw_data

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0


In [3]:
raw_data.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

## Clean dataset

In [4]:
series = pd.isnull(raw_data['cigsPerDay'])


In [5]:
data = raw_data.drop(['currentSmoker','education'], axis = 'columns')
data.head()

,male,age,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [6]:
data.dropna( axis = 0, inplace = True)
data.isnull().sum()

male               0
age                0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [7]:
data.shape

(3749, 14)

## Declare the dependant and independant values


In [8]:
estimators=['male','age','cigsPerDay','BPMeds','prevalentStroke','prevalentHyp','diabetes','totChol','sysBP','glucose']
x1 = data[estimators]
y = data["TenYearCHD"]

In [9]:
x1

,male,age,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,glucose
0,1,39,0.0,0.0,0,0,0,195.0,106.0,77.0
1,0,46,0.0,0.0,0,0,0,250.0,121.0,76.0
2,1,48,20.0,0.0,0,0,0,245.0,127.5,70.0
3,0,61,30.0,0.0,0,1,0,225.0,150.0,103.0
4,0,46,23.0,0.0,0,0,0,285.0,130.0,85.0
...,...,...,...,...,...,...,...,...,...,...
4231,1,58,0.0,0.0,0,1,0,187.0,141.0,81.0
4232,1,68,0.0,0.0,0,1,0,176.0,168.0,79.0
4233,1,50,1.0,0.0,0,1,0,313.0,179.0,86.0
4234,1,51,43.0,0.0,0,0,0,207.0,126.5,68.0


In [10]:
y

0       0
1       0
2       0
3       1
4       0
       ..
4231    0
4232    1
4233    1
4234    0
4237    0
Name: TenYearCHD, Length: 3749, dtype: int64

## Regression

In [11]:
x = sm.add_constant(x1)
reg_logit = sm.Logit(y,x)
results_logit = reg_logit.fit()
results_logit.summary()

Optimization terminated successfully.
         Current function value: 0.377380
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 3749
Model:                          Logit   Df Residuals:                     3738
Method:                           MLE   Df Model:                           10
Date:                Mon, 24 Aug 2020   Pseudo R-squ.:                  0.1165
Time:                        20:04:11   Log-Likelihood:                -1414.8
converged:                       True   LL-Null:                       -1601.4
Covariance Type:            nonrobust   LLR p-value:                 4.802e-74
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -8.7043      0.533    -16.334      0.000      -9.749      -7.660
male                0.5758      0.106      5.452      0.000       0.369       0.783
age                 0.0648      0.006     10.206      0.000       0.052       0.077
cigsPerDay          0.0201      0.004      4.882      0.000       0.012       0.028
BPMeds              0.1560      0.232      0.673      0.501      -0.298       0.610
prevalentStroke     0.7250      0.487      1.489      0.136      -0.229       1.679
prevalentHyp        0.2065      0.134      1.541      0.123      -0.056       0.469
diabetes            0.0195      0.311      0.063      0.950      -0.590       0.629
totChol             0.0022      0.001      2.056      0.040       0.000       0.004
sysBP               0.0140      0.003      4.891      0.000       0.008       0.020
glucose             0.0075      0.002      3.397      0.001       0.003       0.012
===================================================================================
"""

### Confusion Matrix

In [12]:
def confusion_matrix(data,actual_values, model):
    pred_values = model.predict(data) ##predict the values using Logit model
    bins = np.array([0,0.5,1]) ## specify bins
    cm = np.histogram2d(actual_values, pred_values, bins = bins)[0] ## if values are between 0 and 0.5, they will be considered 1. IF values are between 0.5 and 1, they will be considered 1
    accuracy = (cm[0,0]+cm[1,1])/cm.sum()
    return cm, accuracy

In [13]:
cm = confusion_matrix(x,y,results_logit)
cm

(array([[3160.,   17.],
        [ 524.,   48.]]),
 0.8556948519605228)

In [14]:
cm_df = pd.DataFrame(cm[0])
cm_df.columns = ["Predicted 0", "Predicted 1"]
cm_df = cm_df.rename(index={0:"Actual 0", 1: "Actual 1"})
cm_df

,Predicted 0,Predicted 1
Actual 0,3160.0,17.0
Actual 1,524.0,48.0


In [15]:
## For 3160 observations the model predicted 0 and the true value was 0
## For 48 observations the model predicted 1 and the true value was 1... in 3208 cases the model did the job well

## For 524 observations the model predicted 0 while the true value was 1
## For 17 observations the model predicted 1 while the true value was 0... in the 541 cases the model got confused

## Overall the model made an accurate prediction in 3208 out of 3749 cases (85.6% accuracy)

In [16]:
print ("Missclassification rate "+ str((524+17)/3749))

Missclassification rate 0.1443051480394772
